### Student Number : 2
#### Alexandre Lemonnier (alexandre.lemonnier) (23077)
#### Victor Simonin (victor.simonin) (23093)

First, let's import our useful libraries

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, isnull, mean, abs, max, avg, lag, lead, udf, struct, lit, variance, weekofyear, month, year
from pyspark.sql.window import Window
from datetime import timedelta, datetime
from pyspark.ml.stat import Correlation
from sklearn.preprocessing import normalize
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import *
from pyspark.sql import *
from functools import reduce
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

In [21]:
spark_application_name = "Projet_LAVS"
spark = (SparkSession.builder.appName(spark_application_name).getOrCreate())

Here we define the structure of the schema and its columns with their relative types

In [22]:
columns = [StructField("Date",TimestampType()), StructField("High",FloatType()), StructField("Low",FloatType()), StructField("Open",FloatType()), StructField("Close",FloatType()), StructField("Volume",FloatType()), StructField("Adj Close",FloatType()), StructField("company_name",StringType())]
schema = StructType(columns)

def read_data(path):
    return spark.read.schema(schema).csv(path, header=True, sep=',')

Now let's read all the stocks' data :

In [31]:
amazon = read_data('data/AMAZON.csv')
apple = read_data('data/APPLE.csv')
facebook = read_data('data/FACEBOOK.csv')
google = read_data('data/GOOGLE.csv')
microsoft = read_data('data/MICROSOFT.csv')
tesla = read_data('data/TESLA.csv')
zoom = read_data('data/ZOOM.csv')
datasets = [amazon, apple, facebook, google, microsoft, tesla, zoom]

In [24]:
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)
df_stocks = unionAll(*datasets)

In [25]:
df_stocks.columns

['Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close', 'company_name']

In [26]:
df_stocks.summary()

DataFrame[summary: string, High: string, Low: string, Open: string, Close: string, Volume: string, Adj Close: string, company_name: string]

In [27]:
outputPath = "data/stocks.csv"

df_stocks.write.mode("overwrite").csv(outputPath)

In [34]:
trainDF, testDF = df_stocks.randomSplit([.8, .2], seed=42)
print(f"There are {trainDF.cache().count()} rows in the training set, and {testDF.cache().count()} in the test set")

There are 5102 rows in the training set, and 1231 in the test set


In [38]:
trainDF.groupBy("company_name").count().show()

+------------+-----+
|company_name|count|
+------------+-----+
|      AMAZON|  828|
|       APPLE|  794|
|    FACEBOOK|  787|
|      GOOGLE|  793|
|   MICROSOFT|  768|
|       TESLA|  794|
|        ZOOM|  338|
+------------+-----+



In [41]:
testDF.groupBy("company_name").count().show()

+------------+-----+
|company_name|count|
+------------+-----+
|      AMAZON|  159|
|       APPLE|  193|
|    FACEBOOK|  200|
|      GOOGLE|  194|
|   MICROSOFT|  219|
|       TESLA|  193|
|        ZOOM|   73|
+------------+-----+

